<a href="https://colab.research.google.com/github/Niv0902/Shablool/blob/main/ex2_shablul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# 📚 Import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from difflib import get_close_matches
import matplotlib.pyplot as plt
# 📌 NLTK Downloads
nltk.download('stopwords')
nltk.download('wordnet')

# ================================
# 🌐 Build the Index from mqtt.org
# ================================
links = {
    1: "https://mqtt.org/",
    2: "https://mqtt.org/getting-started/",
    3: "https://mqtt.org/software/",
}

custom_stop_words = set(stopwords.words('english'))
custom_stop_words.update([
    'mqtt', 'org', 'https', 'http', 'www', 'com', 'home',
    'page', 'site', 'learn', 'get', 'started', 'software'
])

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text

lemmatizer = WordNetLemmatizer()
index = {}

for doc_id, url in links.items():
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = clean_text(soup.get_text())
    words = text.split()

    for word in words:
        if word in custom_stop_words or len(word) < 3:
            continue
        lemma = lemmatizer.lemmatize(word)
        if lemma not in index:
            index[lemma] = set()
        index[lemma].add(doc_id)

data = {"term": [], "DocIDs": []}
for term, doc_ids in index.items():
    data["term"].append(term)
    data["DocIDs"].append(sorted(list(doc_ids)))

df_index = pd.DataFrame(data)

# ================================
# 🎨 UI Styles (White Background, Black Text)
# ================================
def apply_styles():
    display(HTML("""
    <style>
    body, .output_area {
        background-color: white !important;
        color: black !important;
    }
    .widget-label { font-weight: bold; font-size: 16px; color: black; }
    .widget-button {
        background-color: #4CAF50;
        color: white;
        font-size: 16px;
        border-radius: 8px;
        padding: 8px 16px;
    }
    .widget-text input {
        padding: 8px;
        font-size: 16px;
        border-radius: 8px;
        border: 1px solid #ccc;
        color: black;
        background-color: white;
    }
    </style>
    """))

apply_styles()

# ================================
# 📋 Screens Definitions
# ================================

def show_admin_dashboard():
    clear_output()
    display(main_menu)
    display(HTML("<h2 style='color:black;'>🛠️ Admin Dashboard</h2>"))
    add_term = widgets.Text(description="Add Term:", layout=widgets.Layout(width='400px'))
    add_doc = widgets.Text(description="Add DocID:", layout=widgets.Layout(width='400px'))
    add_button = widgets.Button(description="Add Entry", button_style='success')

    def on_add_click(b):
        term = add_term.value.strip()
        doc = add_doc.value.strip()
        if term and doc:
            if term in df_index['term'].values:
                df_index.loc[df_index['term'] == term, 'DocIDs'].values[0].append(doc)
            else:
                df_index.loc[len(df_index.index)] = [term, [doc]]
            display(HTML("<b style='color:green;'>Entry Added Successfully!</b>"))
            display(df_index)
        else:
            display(HTML("<b style='color:red;'>Please provide both Term and DocID.</b>"))

    add_button.on_click(on_add_click)
    display(add_term, add_doc, add_button)


def show_search_screen():
    clear_output()
    display(main_menu)
    display(HTML("<h2 style='color:black;'>🔍 Search Engine</h2>"))

    search_term = widgets.Text(description="Search Term:", layout=widgets.Layout(width='400px'))
    search_button = widgets.Button(description="Search", button_style='primary')
    smart_search_button = widgets.Button(description="Smart Search", button_style='info')
    results_box = widgets.VBox()

    def display_results(result_df):
        """Display results in styled cards"""
        result_widgets = []
        for _, row in result_df.iterrows():
            card = widgets.HTML(
                value=f"""
                <div style="
                    background-color: #ffffff;
                    padding: 15px;
                    border-radius: 10px;
                    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                    margin-bottom: 10px;
                    color: black;
                    ">
                    <h3 style="color:#4CAF50;">📚 Term: {row['term']}</h3>
                    <p><b>Doc IDs:</b> {', '.join(map(str, row['DocIDs']))}</p>
                </div>
                """
            )
            result_widgets.append(card)
        if result_widgets:
            results_box.children = result_widgets
        else:
            results_box.children = [widgets.HTML("<b style='color:red;'>No results found.</b>")]

    def on_search_click(b):
        term = search_term.value.strip()
        if term:
            result_df = df_index[df_index['term'] == term]
            display_results(result_df)
        else:
            results_box.children = [widgets.HTML("<b style='color:red;'>Please enter a term to search.</b>")]

    def on_smart_search_click(b):
        term = search_term.value.strip()
        matches = get_close_matches(term, df_index['term'].tolist(), n=1, cutoff=0.6)
        if matches:
            corrected = matches[0]
            corrected_df = df_index[df_index['term'] == corrected]
            display(HTML(f"<b>Did you mean '<span style='color:orange;'>{corrected}</span>'?</b>"))
            display_results(corrected_df)
        else:
            results_box.children = [widgets.HTML("<b style='color:red;'>No similar terms found.</b>")]

    search_button.on_click(on_search_click)
    smart_search_button.on_click(on_smart_search_click)

    display(search_term, search_button, smart_search_button, results_box)


def show_statistics_screen():
    clear_output()
    display(main_menu)
    display(HTML("<h2 style='color:black;'>📊 System Statistics Dashboard</h2>"))

    total_terms = len(df_index)
    avg_docs = df_index['DocIDs'].apply(len).mean()
    max_docs = df_index['DocIDs'].apply(len).max()
    unique_docs = set([doc for sublist in df_index['DocIDs'] for doc in sublist])
    total_docs = len(unique_docs)

    stats_html = f"""
    <div style="display: flex; justify-content: space-around; flex-wrap: wrap; font-size:16px; text-align:center; margin-top:20px; color: black;">
        <div style="width: 200px; padding: 15px; background-color:#ffffff; border-radius:10px; margin:10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-size: 24px;">📚</div>
            <div style="font-size: 20px;"><b>{total_terms}</b></div>
            <div>Unique Terms</div>
        </div>
        <div style="width: 200px; padding: 15px; background-color:#ffffff; border-radius:10px; margin:10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-size: 24px;">📄</div>
            <div style="font-size: 20px;"><b>{total_docs}</b></div>
            <div>Unique Documents</div>
        </div>
        <div style="width: 200px; padding: 15px; background-color:#ffffff; border-radius:10px; margin:10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-size: 24px;">📈</div>
            <div style="font-size: 20px;"><b>{round(avg_docs, 2)}</b></div>
            <div>Avg Docs per Term</div>
        </div>
        <div style="width: 200px; padding: 15px; background-color:#ffffff; border-radius:10px; margin:10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
            <div style="font-size: 24px;">🔝</div>
            <div style="font-size: 20px;"><b>{max_docs}</b></div>
            <div>Max Docs per Term</div>
        </div>
    </div>
    """
    display(HTML(stats_html))
    dropdown = widgets.Dropdown(
        options=["Pie Chart", "Bar Chart"],
        value="Pie Chart",
        description="Chart:",
        style={'description_width': '80px'},
        layout=widgets.Layout(width='250px')
    )
    display(dropdown)
    out = widgets.Output()
    display(out)
    def redraw(change):
        with out:
            out.clear_output()
            if change["new"] == "Pie Chart":
                plt.figure(figsize=(6,6))
                doc_counts = df_index['DocIDs'].apply(len).value_counts().sort_index()
                plt.pie(doc_counts, labels=doc_counts.index, autopct='%1.1f%%', startangle=140)
                plt.title('📄 Documents per Term Distribution')
                plt.show()
            if change["new"] == "Bar Chart":
                top_terms = df_index.assign(doc_count=df_index['DocIDs'].apply(len)) \
                                     .sort_values('doc_count', ascending=False) \
                                     .head(10)
                plt.figure(figsize=(8,5))
                plt.bar(top_terms['term'], top_terms['doc_count'])
                plt.xlabel('Term')
                plt.ylabel('Number of Documents')
                plt.title('🔝 Top 10 Terms by Document Count')
                plt.xticks(rotation=45)
                plt.show()
    redraw({"new": dropdown.value})
    dropdown.observe(redraw, names="value")


# ================================
# 🚀 Main Menu with Navigation
# ================================
menu_title = widgets.HTML("<h1 style='text-align:center; color:black;'>📈 Business Statistics Dashboard</h1>")

admin_button = widgets.Button(description="Admin Dashboard",
                              layout=widgets.Layout(width='250px'),
                              button_style='primary')

search_button = widgets.Button(description="Search Engine",
                               layout=widgets.Layout(width='250px'),
                               button_style='primary')

stats_button = widgets.Button(description="Statistics",
                              layout=widgets.Layout(width='250px'),
                              button_style='primary')

admin_button.on_click(lambda b: show_admin_dashboard())
search_button.on_click(lambda b: show_search_screen())
stats_button.on_click(lambda b: show_statistics_screen())

main_menu = widgets.VBox([menu_title, admin_button, search_button, stats_button],
                         layout=widgets.Layout(align_items='center'))


show_admin_dashboard()

Text(value='', description='Add Term:', layout=Layout(width='400px'))

Text(value='', description='Add DocID:', layout=Layout(width='400px'))

Button(button_style='success', description='Add Entry', style=ButtonStyle())